In [1]:
%pip install requests

You should consider upgrading via the 'c:\Users\fenpon\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [2]:
import re
import pandas as pd
import requests


In [18]:


def get_coordinates_kakao(address, api_key):
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {api_key}"}
    params = {"query": address}
    
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        result = response.json()
        if result["documents"]:
            location = result["documents"][0]["address"]
            if location is None:
                return None, None
            return float(location["y"]), float(location["x"])  # 위도, 경도 반환
        else:
            print(f"해당 주소에 대한 좌표 정보를 찾을 수 없습니다. {address}")
            return None, None
    else:
        print( f"에러 발생: {response.status_code}")
        return None, None


In [ ]:

#병원 점수 예측 데이터
trainedHospital = pd.read_csv('../data/safety/encoded-소방청_시도 소방서 현황_20240630.csv')


block = 0
le = 0
# 사용 예제


for i in range(len(trainedHospital)):
    address = trainedHospital.iloc[i]['주소']
    address = address.split(' ')
    address = ' '.join(address[:4])
    coord = get_coordinates_kakao(address,'api key')
    trainedHospital.at[i,'lat'] = coord[0]
    trainedHospital.at[i,'lon'] = coord[1]

    block += 1
    if block > 100:
        print("--- " ,block , "  : ",le)
        block = 0
        le += 1

---  101   :  0
해당 주소에 대한 좌표 정보를 찾을 수 없습니다. 경상북도 구미소방서 수출대로112
---  101   :  1


In [22]:
trainedHospital.to_csv('../export/safety/encoded-소방청_시도 소방서 현황_20240630_convert.csv', index=False, encoding='utf-8') 
